# **Imports**

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!pip install leidenalg
!pip install cairocffi

import leidenalg
import igraph as ig
import numpy as np
import torch
import random
import pickle
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.io import loadmat
import os
import time
import json
from tqdm import tqdm
from os.path import join
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import networkx as nx


random.seed(1234)



class arguments:
    dataset = "Ciao"
    test_prop = 0.1
    enable_improvement = True

args = arguments()

In [ ]:
# path to datasets
workdir = '/content/gdrive/MyDrive/RS_prog/datasets/'

# path for model check points saving
workdir_checkpoints_save = "/content/gdrive/MyDrive/RS_prog/checkpoints/"

In [ ]:
# load dataset
if args.dataset == 'Ciao':
	click_f = loadmat(workdir + 'Ciao/rating.mat')['rating']
	trust_f = loadmat(workdir + 'Ciao/trustnetwork.mat')['trustnetwork']
elif args.dataset == 'Epinions':
	click_f = np.loadtxt(workdir+'Epinions/ratings_data.txt', dtype = np.int32)
	trust_f = np.loadtxt(workdir+'Epinions/trust_data.txt', dtype = np.int32)

In [ ]:
trust_f[456]

# **Preprocessing**

In [ ]:

# load dataset
if args.dataset == 'Ciao':
	click_f = loadmat(workdir + 'Ciao/rating.mat')['rating']
	trust_f = loadmat(workdir + 'Ciao/trustnetwork.mat')['trustnetwork']
elif args.dataset == 'Epinions':
	click_f = np.loadtxt(workdir+'Epinions/ratings_data.txt', dtype = np.int32)
	trust_f = np.loadtxt(workdir+'Epinions/trust_data.txt', dtype = np.int32)
else:
	pass 

click_list = []
trust_list = []

u_items_list = []
u_users_list = []
u_users_items_list = []
i_users_list = []

pos_u_items_list = []
pos_i_users_list = []

############### added for improvment
u_users_items_list_group = []
u_users_list_group = []
###############

user_count = 0
item_count = 0
rate_count = 0

# read <user, item, rating> file, and create a list of arrays(each one represents a sample)
for s in click_f:
	uid = s[0]
	iid = s[1]
	if args.dataset == 'Ciao':
		label = s[3]
	elif args.dataset == 'Epinions':
		label = s[2]

	if uid > user_count:
		user_count = uid
	if iid > item_count:
		item_count = iid
	if label > rate_count:
		rate_count = label
	click_list.append([uid, iid, label])


pos_list = []
for i in range(len(click_list)):
	pos_list.append((click_list[i][0], click_list[i][1], click_list[i][2]))

# remove duplicate items in pos_list because there are some cases where a user may have different rate scores on the same item.
pos_list = list(set(pos_list))

# filter user less than 5 items
pos_df = pd.DataFrame(pos_list, columns = ['uid', 'iid', 'label'])
filter_pos_list = []
user_in_set, user_out_set = set(), set()
for u in range(user_count + 1):
	hist = pos_df[pos_df['uid'] == u]
	if len(hist) < 5:
		user_out_set.add(u)
		continue
	user_in_set.add(u)
	u_items = hist['iid'].tolist()
	u_ratings = hist['label'].tolist()
	filter_pos_list.extend([(u, iid, rating) for iid, rating in zip(u_items, u_ratings)])
# print('user in and out size: ', len(user_in_set), len(user_out_set))


db_user_before_filtering = len(pos_df['uid'].unique())
print('No users before: ', db_user_before_filtering,' , after: ', len( user_in_set))
print('No. Ratings before:', len(pos_list),' and after filtering: ', len(filter_pos_list))
db_items_before_filtering = len(pos_df['uid'].unique())
ratings_density_before = len(pos_list) / (db_user_before_filtering * db_items_before_filtering)


# train, valid and test data split
print('test prop: ', args.test_prop)
pos_list = filter_pos_list
random.shuffle(pos_list)
num_test = int(len(pos_list) * args.test_prop)
test_set = pos_list[:num_test]
valid_set = pos_list[num_test:2 * num_test]
train_set = pos_list[2 * num_test:]
print('Train samples: {}, Valid samples: {}, Test samples: {}, Total samples: {}'.format(len(train_set), len(valid_set), len(test_set), len(pos_list)))

# save splitted train, validaition, test sets to folder
with open(workdir + args.dataset + '/dataset_filter5_'+str(args.test_prop)+'.pkl', 'wb') as f:
	pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(valid_set, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)


# convert the splitted sets to DataFrames
pos_df = pd.DataFrame(pos_list, columns = ['uid', 'iid', 'label'])
train_df = pd.DataFrame(train_set, columns = ['uid', 'iid', 'label'])
valid_df = pd.DataFrame(valid_set, columns = ['uid', 'iid', 'label'])
test_df = pd.DataFrame(test_set, columns = ['uid', 'iid', 'label'])

click_df = pd.DataFrame(click_list, columns = ['uid', 'iid', 'label'])

# sort values by user_id
train_df = train_df.sort_values(axis = 0, ascending = True, by = 'uid')
pos_df = pos_df.sort_values(axis = 0, ascending = True, by = 'uid')

"""
creates list (u_items_list) of arrays. Each array contains tuples of (item id, rating), belongs to specific user. 
if user didn't have rating items the following tuple [(0, 0)] is added. 
"""
for u in range(user_count + 1):
	hist = train_df[train_df['uid'] == u]
	u_items = hist['iid'].tolist()
	u_ratings = hist['label'].tolist()
	if u_items == []:
		u_items_list.append([(0, 0)])
	else:
		u_items_list.append([(iid, rating) for iid, rating in zip(u_items, u_ratings)])


"""
creates list (i_users_list) of arrays. Each array contains tuples of (user id, rating), belongs to specific item. 
if item didn't rated by any user the following tuple [(0, 0)] is added. 
"""
train_df = train_df.sort_values(axis = 0, ascending = True, by = 'iid')

userful_item_set = set()
for i in range(item_count + 1):
	hist = train_df[train_df['iid'] == i]
	i_users = hist['uid'].tolist()
	i_ratings = hist['label'].tolist()
	if i_users == []:
		i_users_list.append([(0, 0)])
	else:
		i_users_list.append([(uid, rating) for uid, rating in zip(i_users, i_ratings)])
		userful_item_set.add(i)

print('item size before: ', item_count,"  and after filtering: ", len(userful_item_set))
ratings_density_after = len(pos_list)/(len(user_in_set)*len(userful_item_set))


print('Ratings Density before: ', ratings_density_before , ' , after: ', ratings_density_after)

# save postprocessed user-item-rating data files
with open(workdir + args.dataset + '/effective_users_and items_filter5.pkl', 'wb') as f:
	pickle.dump(list(user_in_set), f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(list(userful_item_set), f, pickle.HIGHEST_PROTOCOL)
 
"""
processing of social relations file, reads data to list that represents egdes
of user nodes connections, [A,B] means user A follow after user B. 
 """
count_f_origin, count_f_filter = 0,0
for s in trust_f:
	uid = s[0]
	fid = s[1]
	count_f_origin += 1
	if uid > user_count or fid > user_count:
		continue
	if uid in user_out_set or fid in user_out_set:
		continue
	trust_list.append([uid, fid])
	count_f_filter += 1

print('No. of social connections before: ' , count_f_origin, ' ,after: ', count_f_filter)
social_density_after = count_f_filter/(len(user_in_set)**2)
social_density_before = count_f_origin/(db_user_before_filtering**2)
print('social density before: ',   social_density_before , ', after:', social_density_after)
trust_df = pd.DataFrame(trust_list, columns = ['uid', 'fid'])
trust_df = trust_df.sort_values(axis = 0, ascending = True, by = 'uid')


"""
u_users_list: list of users contains for each one aggregation of users that he follows, if he didnt follow any user [0] added.  
u_users_items_list: list of users contains for each one list of list represents items and rating of following users.
if user didnt follow any user [0,0] added.
"""
count_0, count_1 = 0,0
for u in range(user_count + 1):
	hist = trust_df[trust_df['uid'] == u]
	u_users = hist['fid'].unique().tolist()
	if u_users == []:
		u_users_list.append([0])
		u_users_items_list.append([[(0,0)]])
		count_0 += 1
	else:
		u_users_list.append(u_users)
		uu_items = []
		for uid in u_users:
			uu_items.append(u_items_list[uid])
		u_users_items_list.append(uu_items)
		count_1 += 1

print('trust user with items size: ', count_0, count_1)




No users before:  40163  , after:  23253
No. Ratings before: 664824  and after filtering:  631192
test prop:  0.1
Train samples: 504954, Valid samples: 63119, Test samples: 63119, Total samples: 631192
item size before:  139738   and after filtering:  120711
Ratings Density before:  0.00041214914051076584  , after:  0.00022487213666856555
No. of social connections before:  487183  ,after:  374039
social density before:  0.0003020228733039969 , after: 0.0006917655081248274
trust user with items size:  31405 17885


## **Communities detection - Leidenalg**

In [ ]:

# load social graph egdes
tuples = [tuple(x) for x in trust_df.values]
g3 = ig.Graph.TupleList(tuples, directed = True)

# partition of social graph to communities 
part = leidenalg.find_partition(g3, leidenalg.SignificanceVertexPartition, n_iterations=-1, seed=42, max_comm_size=200)

com_list = [com for com in part if len(com) > 2]
with open('/content/gdrive/MyDrive/RS_prog/comm_list_'+args.dataset+'.pkl', 'wb') as fp:
    pickle.dump(com_list, fp)

# read communities file, after prepare using a communities detection algorithm
with open ('/content/gdrive/MyDrive/RS_prog/comm_list_'+args.dataset+'.pkl', 'rb') as fp:
    comm_list = pickle.load(fp)

## **Preprocessing Improvement** 

In [ ]:
# improvement group list ###################################################
dict_users_groups = {}

# create for each user list of his community users 
for scc in comm_list:
    for person in scc:
      list_scc = scc.copy()
      list_scc.remove(person)
      dict_users_groups[person] = list_scc

"""
preparing data of communities
u_users_list_group: list of users contains for each one aggregation of users that part of his community, if he doesnt part of community [0] added.  
u_users_items_list_group: list of users contains for each one list of list represents items and rating of his community users.
if user didnt part of any community [0,0] added.
"""
for u in range(user_count + 1):
    u_users = []
    if u in dict_users_groups:

        u_users = dict_users_groups[u]

    if u_users == []:
        u_users_list_group.append([0])
        u_users_items_list_group.append([[(0,0)]])
    else:
        u_users_list_group.append(u_users)
        uu_items = []
        for uid in u_users:
          uu_items.append(u_items_list[uid])
        u_users_items_list_group.append(uu_items)

######################### end preprocessing - improvment part


## **Save processed data**

In [ ]:
# save preprocessing data
with open(workdir + args.dataset + '/list_filter5_'+str(args.test_prop)+'.pkl', 'wb') as f:
      pickle.dump(u_items_list, f, pickle.HIGHEST_PROTOCOL)
      pickle.dump(u_users_list, f, pickle.HIGHEST_PROTOCOL)
      pickle.dump(u_users_items_list, f, pickle.HIGHEST_PROTOCOL)
      pickle.dump(i_users_list, f, pickle.HIGHEST_PROTOCOL)
      pickle.dump((user_count, item_count, rate_count), f, pickle.HIGHEST_PROTOCOL)
      pickle.dump(u_users_list_group, f, pickle.HIGHEST_PROTOCOL)        
      pickle.dump(u_users_items_list_group, f, pickle.HIGHEST_PROTOCOL)

# **Utils**

In [ ]:

truncate_len = 50
truncate_len_i = 10

"""
Ciao dataset info:
Avg number of items rated per user: 38.3
Avg number of users interacted per user: 2.7
Avg number of users connected per item: 16.4
"""

def collate_fn(batch_data):
    """
    This function will be used to pad the graph to max length in the batch.
    It will be used in the Dataloader.
    """
    uids, iids, labels = [], [], []
    u_items, u_users, u_users_items, i_users, u_users_group, u_users_items_group  = [], [], [], [], [], []
    u_items_len, u_users_len, i_users_len, u_users_len_group = [], [], [], []
    count = 0
    for data, u_items_u, u_users_u, u_users_items_u, i_users_i, u_users_u_group, u_users_items_u_group in batch_data:

        (uid, iid, label) = data
        uids.append(uid)
        iids.append(iid)
        labels.append(label)

        # user-items    
        if len(u_items_u) <= truncate_len:
            u_items.append(u_items_u)
        else:
            u_items.append(random.sample(u_items_u, truncate_len))
        u_items_len.append(min(len(u_items_u), truncate_len))


        # user-users and user-users-items
        if len(u_users_u) < truncate_len:
            tmp_users = [item for item in u_users_u]
            tmp_users.append(uid)
            u_users.append(tmp_users)
            u_u_items = [] 
            for uui in u_users_items_u:
                if len(uui) < truncate_len:
                    u_u_items.append(uui)
                else:
                    u_u_items.append(random.sample(uui, truncate_len))
            # self -loop
            u_u_items.append(u_items[-1])
            u_users_items.append(u_u_items)

        else:
            sample_index = random.sample(list(range(len(u_users_u))), truncate_len-1)
            tmp_users = [u_users_u[si] for si in sample_index]
            tmp_users.append(uid)
            u_users.append(tmp_users)

            u_users_items_u_tr = [u_users_items_u[si] for si in sample_index]
            u_u_items = [] 
            for uui in u_users_items_u_tr:
                if len(uui) < truncate_len:
                    u_u_items.append(uui)
                else:
                    u_u_items.append(random.sample(uui, truncate_len))
            u_u_items.append(u_items[-1])
            u_users_items.append(u_u_items)

        u_users_len.append(min(len(u_users_u)+1, truncate_len))

        # item-users
        if len(i_users_i) <= truncate_len:
            i_users.append(i_users_i)
        else:
            i_users.append(random.sample(i_users_i, truncate_len))
        i_users_len.append(min(len(i_users_i), truncate_len))

################################################################################# added for improvement

         # user-users group and user-users-items group
        if len(u_users_u_group) <= truncate_len:
            u_users_group.append(u_users_u_group)
            u_u_items_group = [] 
            for uui_group in u_users_items_u_group:
                if len(uui_group) < truncate_len:
                    u_u_items_group.append(uui_group)
                else:
                    u_u_items_group.append(random.sample(uui_group, truncate_len))
            u_users_items_group.append(u_u_items_group)
        else:
            sample_index = random.sample(list(range(len(u_users_u_group))), truncate_len)
            u_users_group.append([u_users_u_group[si] for si in sample_index])

            u_users_items_u_tr_group = [u_users_items_u_group[si] for si in sample_index]
            u_u_items_group = [] 
            for uui_group in u_users_items_u_tr_group:
                if len(uui_group) < truncate_len:
                    u_u_items_group.append(uui_group)
                else:
                    u_u_items_group.append(random.sample(uui_group, truncate_len))
            u_users_items_group.append(u_u_items_group)

        u_users_len_group.append(truncate_len)	 #(min(len(u_users_u_group), truncate_len))	

################################################################################## until here
    batch_size = len(batch_data)

    # padding size
    u_items_maxlen = max(u_items_len)
    u_users_maxlen = max(u_users_len)
    i_users_maxlen = max(i_users_len)

    
    u_item_pad = torch.zeros([batch_size, u_items_maxlen, 2], dtype=torch.long)
    for i, ui in enumerate(u_items):
        u_item_pad[i, :len(ui), :] = torch.LongTensor(ui)
    
    u_user_pad = torch.zeros([batch_size, u_users_maxlen], dtype=torch.long)
    for i, uu in enumerate(u_users):
        u_user_pad[i, :len(uu)] = torch.LongTensor(uu)


    u_user_item_pad = torch.zeros([batch_size, u_users_maxlen, u_items_maxlen, 2], dtype=torch.long)
    for i, uu_items in enumerate(u_users_items):
        for j, ui in enumerate(uu_items):
            u_user_item_pad[i, j, :len(ui), :] = torch.LongTensor(ui)

    i_user_pad = torch.zeros([batch_size, i_users_maxlen, 2], dtype=torch.long)
    for i, iu in enumerate(i_users):
        i_user_pad[i, :len(iu), :] = torch.LongTensor(iu)

################## added for improvement
    u_users_maxlen_group = max(u_users_len_group)

    u_user_pad_group = torch.zeros([batch_size, u_users_maxlen_group], dtype=torch.long)
    for i, uu in enumerate(u_users_group):
        u_user_pad_group[i, :len(uu)] = torch.LongTensor(uu)

    u_user_item_pad_group = torch.zeros([batch_size, u_users_maxlen_group, u_items_maxlen, 2], dtype=torch.long)
    for i, uu_items in enumerate(u_users_items_group):
        for j, ui in enumerate(uu_items):
            u_user_item_pad_group[i, j, :len(ui), :] = torch.LongTensor(ui)
####################

    return torch.LongTensor(uids), torch.LongTensor(iids), torch.FloatTensor(labels), \
            u_item_pad, u_user_pad, u_user_item_pad, i_user_pad, u_user_pad_group, u_user_item_pad_group

# **Data Loader**

In [ ]:
class GRDataset(Dataset):
    """
    This class provides an envelope for iterating over the data.
    """
    def __init__(self, data, u_items_list, u_users_list, u_users_items_list, i_users_list, u_users_list_group, u_users_items_list_group):
          """
          Initializing the class instance.
          """
          self.data = data
          self.u_items_list = u_items_list
          self.u_users_list = u_users_list
          self.u_users_items_list = u_users_items_list
          self.i_users_list = i_users_list
          self.u_users_list_group = u_users_list_group
          self.u_users_items_list_group = u_users_items_list_group

    def __getitem__(self, index):
          """
          get all graph info relevent for data in given index. 
          """
          uid = self.data[index][0]
          iid = self.data[index][1]
          label = self.data[index][2]
          u_items = self.u_items_list[uid]
          u_users = self.u_users_list[uid]
          u_users_items = self.u_users_items_list[uid]
          i_users = self.i_users_list[iid]

          ###################
          u_users_group = self.u_users_list_group[uid]
          u_users_items_group = self.u_users_items_list_group[uid]
          ###################

          return (uid, iid, label), u_items, u_users, u_users_items, i_users, u_users_group, u_users_items_group

    def __len__(self):
          """
          get data length.
          """
          return len(self.data)

# **Model**

In [ ]:
class _MultiLayerPercep(nn.Module):
    """
    Auxiliary class of MLP functionality.
    This structure called from GraphRec structure.
    """
    def __init__(self, input_dim, output_dim):
        """
        Initializition of MLP structure.

        Parameters: 
        input_dim - input size for MLP network
        output_dim - output size for MLP network
        """
        super(_MultiLayerPercep, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2, bias=True),
            nn.LeakyReLU(0.2),
            nn.Linear(input_dim // 2, output_dim, bias=True),
        )

    def forward(self, x):
        """
        MLP forward functionality
        """
        return self.mlp(x)


class _Aggregation(nn.Module):
    """
    Auxiliary class for items/users aggregation.
    This structure called from GraphRec structure.
    """
    def __init__(self, input_dim, output_dim):
        """
        Initializition of aggregation structure.

        Parameters: 
        input_dim - input size for neural network
        output_dim - output size for neural network

        """
        super(_Aggregation, self).__init__()
        self.aggre = nn.Sequential(
            nn.Linear(input_dim, output_dim, bias=True),
            nn.ReLU(),
        )

    def forward(self, x):
        """
        Neural network forward functionality
        """
        return self.aggre(x)


class _UserModel(nn.Module):
    ''' 
    User modeling to learn user latent factors.
    User modeling leverages two types aggregation: item aggregation and social aggregation

    social aggregation built from two information sets:
    1. trust graph (how each user follow)
    2. communities graph (community information)
    '''
    def __init__(self, emb_dim, user_emb, item_emb, rate_emb):

        super(_UserModel, self).__init__()
        self.user_emb = user_emb
        self.item_emb = item_emb
        self.rate_emb = rate_emb
        self.emb_dim = emb_dim

        self.w1 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w2 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w3 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w4 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w5 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w6 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w7 = nn.Linear(self.emb_dim, self.emb_dim)

        #### added

        self.w8 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w9 = nn.Linear(self.emb_dim, self.emb_dim)
        self.g_v_social = _MultiLayerPercep(2 * self.emb_dim, self.emb_dim)
        ####


        self.g_v = _MultiLayerPercep(2 * self.emb_dim, self.emb_dim)

        self.user_items_att = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.aggre_items = _Aggregation(self.emb_dim, self.emb_dim)

        self.user_items_att_s1 = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.aggre_items_s1 = _Aggregation(self.emb_dim, self.emb_dim)
        self.user_users_att_s2 = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.aggre_neigbors_s2 = _Aggregation(self.emb_dim, self.emb_dim)

        self.u_user_users_att = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.u_aggre_neigbors = _Aggregation(self.emb_dim, self.emb_dim)

        #### added for improvement
        self.user_users_att_group = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.aggre_neigbors_group = _Aggregation(self.emb_dim, self.emb_dim)


        self.combine_mlp_social = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(2 * self.emb_dim, 2*self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(2*self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU()
        )
        ####

        self.combine_mlp = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(2 * self.emb_dim, 2*self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(2*self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU()
        )

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # used for preventing zero div error when calculating softmax score
        self.eps = 1e-10

    def forward(self, uids, u_item_pad, u_user_pad, u_user_item_pad, u_user_pad_group, u_user_item_pad_group):
        """
        User modeling part, forward information pass.
        Returns user latent space. 
        """
        # item aggregation
        q_a = self.item_emb(u_item_pad[:,:,0])   # B x maxi_len x emb_dim
        mask_u = torch.where(u_item_pad[:,:,0] > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))   # B x maxi_len
        u_item_er = self.rate_emb(u_item_pad[:, :, 1])  # B x maxi_len x emb_dim
        x_ia = self.g_v(torch.cat([q_a, u_item_er], dim=2).view(-1, 2 * self.emb_dim)).view(q_a.size())  # B x maxi_len x emb_dim
        p_i = mask_u.unsqueeze(2).expand_as(q_a) * self.user_emb(uids).unsqueeze(1).expand_as(q_a)  # B x maxi_len x emb_dim

        alpha = self.user_items_att(torch.cat([self.w1(x_ia), self.w1(p_i)], dim = 2).view(-1, 2 * self.emb_dim)).view(mask_u.size()) # B x maxi_len
        alpha = torch.exp(alpha) * mask_u
        alpha = alpha / (torch.sum(alpha, 1).unsqueeze(1).expand_as(alpha) + self.eps)

        h_iI = self.aggre_items(torch.sum(alpha.unsqueeze(2).expand_as(x_ia) * x_ia, 1))     # B x emb_dim
        h_iI = F.dropout(h_iI, 0.5, training=self.training)



        # social aggregation
        q_a_s = self.item_emb(u_user_item_pad[:,:,:,0])   # B x maxu_len x maxi_len x emb_dim
        mask_s = torch.where(u_user_item_pad[:,:,:,0] > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))  # B x maxu_len x maxi_len
        p_i_s = mask_s.unsqueeze(3).expand_as(q_a_s) * self.user_emb(u_user_pad).unsqueeze(2).expand_as(q_a_s)  # B x maxu_len x maxi_len x emb_dim
        u_user_item_er = self.rate_emb(u_user_item_pad[:, :, :, 1])  # B x maxu_len x maxi_len x emb_dim
        x_ia_s = self.g_v(torch.cat([q_a_s, u_user_item_er], dim=3).view(-1, 2 * self.emb_dim)).view(q_a_s.size())  # B x maxu_len x maxi_len x emb_dim

        alpha_s = self.user_items_att_s1(torch.cat([self.w4(x_ia_s), self.w4(p_i_s)], dim = 3).view(-1, 2 * self.emb_dim)).view(mask_s.size())    # B x maxu_len x maxi_len
        alpha_s = torch.exp(alpha_s) * mask_s
        alpha_s = alpha_s / (torch.sum(alpha_s, 2).unsqueeze(2).expand_as(alpha_s) + self.eps)


        h_oI_temp = torch.sum(alpha_s.unsqueeze(3).expand_as(x_ia_s) * x_ia_s, 2)    # B x maxu_len x emb_dim
        h_oI = self.aggre_items_s1(h_oI_temp.view(-1, self.emb_dim)).view(h_oI_temp.size())  # B x maxu_len x emb_dim
        h_oI = F.dropout(h_oI, p=0.5, training=self.training)

        ## calculate attention scores in social aggregation
        mask_su = torch.where(u_user_pad > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))

        beta = self.user_users_att_s2(torch.cat([self.w5(h_oI), self.w5(self.user_emb(u_user_pad))], dim = 2).view(-1, 2 * self.emb_dim)).view(u_user_pad.size())
        beta = torch.exp(beta) * mask_su
        beta = beta / (torch.sum(beta, 1).unsqueeze(1).expand_as(beta) + self.eps)
        h_iS = self.aggre_neigbors_s2(torch.sum(beta.unsqueeze(2).expand_as(h_oI) * h_oI, 1))     # B x emb_dim
        h_iS = F.dropout(h_iS, p=0.5, training=self.training)

        if args.enable_improvement:
            # group social aggregation
            q_a_s_group = self.item_emb(u_user_item_pad_group[:,:,:,0])   # B x maxu_len x maxi_len x emb_dim
            mask_s_group = torch.where(u_user_item_pad_group[:,:,:,0] > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))  # B x community_maxu_len x maxi_len
            p_i_s_group = mask_s_group.unsqueeze(3).expand_as(q_a_s_group) * self.user_emb(u_user_pad_group).unsqueeze(2).expand_as(q_a_s_group)  # B x community_maxu_len x maxi_len x emb_dim
            u_user_item_er_group = self.rate_emb(u_user_item_pad_group[:,:,:,1]) # B x maxu_len x maxi_len x emb_dim
            x_ia_s_group = self.g_v_social(torch.cat([q_a_s_group, u_user_item_er_group], dim = 3).view(-1, 2 * self.emb_dim)).view(q_a_s_group.size())  # B x community_maxu_len x maxi_len x emb_dim   


            alpha_s_group = self.user_items_att(torch.cat([self.w8(x_ia_s_group), self.w8(p_i_s_group)], dim = 3).view(-1, 2 * self.emb_dim)).view(mask_s_group.size())    # B x community_maxu_len x maxi_len
            alpha_s_group = torch.exp(alpha_s_group) * mask_s_group
            alpha_s_group = alpha_s_group / (torch.sum(alpha_s_group, 2).unsqueeze(2).expand_as(alpha_s_group) + self.eps)

            h_oI_temp_group = torch.sum(alpha_s_group.unsqueeze(3).expand_as(x_ia_s_group) * x_ia_s_group, 2)    # B x community_maxu_len x emb_dim
            h_oI_group = self.aggre_items(h_oI_temp_group.view(-1, self.emb_dim)).view(h_oI_temp_group.size())  # B x community_maxu_len x emb_dim

            mask_su_group = torch.where(u_user_pad_group > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))

            ## group - calculate attention scores in social aggregation
            beta_group = self.user_users_att_group(torch.cat([self.w9(h_oI_group), self.w9(self.user_emb(u_user_pad_group))], dim = 2).view(-1, 2 * self.emb_dim)).view(u_user_pad_group.size())
            beta_group = torch.exp(beta_group) * mask_su_group
            beta_group = beta_group / (torch.sum(beta_group, 1).unsqueeze(1).expand_as(beta_group) + self.eps)
            h_iS_group = self.aggre_neigbors_group(torch.sum(beta.unsqueeze(2).expand_as(h_oI_group) * h_oI_group, 1))     # B x emb_dim
            h_iS_group = F.dropout(h_iS_group, p=0.5, training=self.training)


            ##  combine community and trust(friends) spaces
            h_iS = self.combine_mlp_social(torch.cat([h_iS_group, h_iS], dim = 1))


        ## learning user latent factor
        h =  self.combine_mlp(torch.cat([h_iI, h_iS], dim = 1))

        return h


class _ItemModel(nn.Module):
    '''Item modeling to learn item latent factors.
    '''
    def __init__(self, emb_dim, user_emb, item_emb, rate_emb):
        super(_ItemModel, self).__init__()
        self.emb_dim = emb_dim
        self.user_emb = user_emb
        self.item_emb = item_emb
        self.rate_emb = rate_emb

        self.w1 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w2 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w3 = nn.Linear(self.emb_dim, self.emb_dim)
        self.w4 = nn.Linear(self.emb_dim, self.emb_dim)

        self.g_u = _MultiLayerPercep(2 * self.emb_dim, self.emb_dim)
        self.g_v = _MultiLayerPercep(2 * self.emb_dim, self.emb_dim)

        self.item_users_att_i = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.aggre_users_i = _Aggregation(self.emb_dim, self.emb_dim)

        self.item_users_att = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.aggre_users = _Aggregation(self.emb_dim, self.emb_dim)

        self.i_friends_att = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.aggre_i_friends = _Aggregation(self.emb_dim, self.emb_dim)

        self.if_friends_att = _MultiLayerPercep(2 * self.emb_dim, 1)
        self.aggre_if_friends = _Aggregation(self.emb_dim, self.emb_dim)

        self.combine_mlp = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(3* self.emb_dim, 2*self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(2*self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU()
        )

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # used for preventing zero div error when calculating softmax score
        self.eps = 1e-10

    def forward(self, iids, i_user_pad):
        """
        Item modeling part, forward information pass.
        Returns item latent space. 
        """
        # user aggregation
        p_t = self.user_emb(i_user_pad[:,:,0])
        mask_i = torch.where(i_user_pad[:,:,0] > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        i_user_er = self.rate_emb(i_user_pad[:,:,1])
        f_jt = self.g_u(torch.cat([p_t, i_user_er], dim = 2).view(-1, 2 * self.emb_dim)).view(p_t.size())
        
        # calculate attention scores in user aggregation
        q_j = mask_i.unsqueeze(2).expand_as(f_jt) * self.item_emb(iids).unsqueeze(1).expand_as(f_jt)
        
        miu = self.item_users_att_i(torch.cat([self.w1(f_jt), self.w1(q_j)], dim = 2).view(-1, 2 * self.emb_dim)).view(mask_i.size())
        miu = torch.exp(miu) * mask_i
        miu = miu / (torch.sum(miu, 1).unsqueeze(1).expand_as(miu) + self.eps)
        z_j = self.aggre_users_i(torch.sum(miu.unsqueeze(2).expand_as(f_jt) * self.w1(f_jt), 1))
        z_j = F.dropout(z_j, p=0.5, training=self.training)


        return z_j


class GraphRec(nn.Module):
    '''
    GraphRec model proposed in the paper Graph neural network for social recommendation 
    Args:
        number_users: the number of users in the dataset.
        number_items: the number of items in the dataset.
        num_rate_levels: the number of rate levels in the dataset.
        emb_dim: the dimension of user and item embedding (default = 64).
    '''
    def __init__(self, num_users, num_items, num_rate_levels, emb_dim = 64):
        super(GraphRec, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.num_rate_levels = num_rate_levels
        self.emb_dim = emb_dim
        self.user_emb = nn.Embedding(self.num_users, self.emb_dim, padding_idx = 0)
        self.item_emb = nn.Embedding(self.num_items, self.emb_dim, padding_idx = 0)
        self.rate_emb = nn.Embedding(self.num_rate_levels, self.emb_dim, padding_idx = 0)

        self.user_model = _UserModel(self.emb_dim, self.user_emb, self.item_emb, self.rate_emb)

        self.item_model = _ItemModel(self.emb_dim, self.user_emb, self.item_emb, self.rate_emb)

        
        self.rate_pred = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(2* self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(self.emb_dim, self.emb_dim // 4),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(self.emb_dim // 4, 1)
        )



    def forward(self, uids, iids, u_item_pad, u_user_pad, u_user_item_pad, i_user_pad, u_user_pad_group, u_user_item_pad_group):
        '''
        Args:
            uids: the user id sequences.
            iids: the item id sequences.
            u_item_pad: the padded user-item graph.
            u_user_pad: the padded user-user graph.
            u_user_item_pad: the padded user-user-item graph.
            i_user_pad: the padded item-user graph.
            u_user_pad_group:the padded user-community(user) graph.
            u_user_item_pad_group:the padded user-community(user)-item graph.
        Shapes:
            uids: (B).
            iids: (B).
            u_item_pad: (B, ItemSeqMaxLen, 2).
            u_user_pad: (B, UserSeqMaxLen).
            u_user_item_pad: (B, UserSeqMaxLen, ItemSeqMaxLen, 2).
            i_user_pad: (B, UserSeqMaxLen, 2).
            u_user_pad_group: (B, CommunityUserSeqMaxLen).
            u_user_item_pad_group: (B, CommunityUserSeqMaxLen, ItemSeqMaxLen, 2).
        Returns:
            the predicted rate scores of the user to the item.
        '''

        h = self.user_model(uids, u_item_pad, u_user_pad, u_user_item_pad, u_user_pad_group, u_user_item_pad_group)
        z = self.item_model(iids, i_user_pad)

        r_ij = self.rate_pred(torch.cat([h,z], dim = 1))

        return r_ij


# **Main**

In [ ]:
# run parameters
dataset_path = workdir + args.dataset +"/"
batch_size = 128
embed_dim = 64
args_epoch = 30
seed = 1234
lr = 0.001
lr_dc = 0.5
lr_dc_step = 30



# set seed and run configuration: cpu/gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)


# set location for training weights checkpoints
fn = workdir_checkpoints_save 
if not os.path.exists(fn):
    os.mkdir(fn)

fn = workdir_checkpoints_save + args.dataset +'_'+str(args.test_prop)+"/" 
if not os.path.exists(fn):
    os.mkdir(fn)


# main function for train/test 
def main(test =  False ):

    
    # load train, validation, test sets
    print('Loading data...')
    with open(dataset_path + 'dataset_filter5_'+str(args.test_prop)+'.pkl', 'rb') as f:
        train_set = pickle.load(f)
        valid_set = pickle.load(f)
        test_set = pickle.load(f)


    # load preprocessed information for graphs
    with open(dataset_path + 'list_filter5_'+str(args.test_prop)+'.pkl', 'rb') as f:
        u_items_list = pickle.load(f)
        u_users_list = pickle.load(f)
        u_users_items_list = pickle.load(f)
        i_users_list = pickle.load(f)
        (user_count, item_count, rate_count) = pickle.load(f)
        u_users_list_group = pickle.load(f)
        u_users_items_list_group = pickle.load(f)



    # prepare data as torch DataLoader 
    train_data = GRDataset(train_set, u_items_list,  u_users_list, u_users_items_list, i_users_list,u_users_list_group,u_users_items_list_group)
    valid_data = GRDataset(valid_set, u_items_list,  u_users_list, u_users_items_list, i_users_list,u_users_list_group,u_users_items_list_group)
    test_data = GRDataset(test_set, u_items_list, u_users_list,  u_users_items_list, i_users_list,u_users_list_group,u_users_items_list_group)
    train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True, collate_fn = collate_fn)
    valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = False, collate_fn = collate_fn)
    test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False, collate_fn = collate_fn)


    model = GraphRec(user_count+1, item_count+1, rate_count+1, embed_dim).to(device)

    # if it's test run, run model on test data, and print rmse and mae score.
    if test:
        print('Load checkpoint and testing...')
        ckpt = torch.load('%s/random_best_checkpoint.pth.tar' %fn)
        model.load_state_dict(ckpt['state_dict'])
        mae, rmse = validate(test_loader, model)
        print("Test: MAE: {:.4f}, RMSE: {:.4f}".format(mae, rmse))
        return

def trainForEpoch(train_loader, model, optimizer, epoch, num_epochs, criterion, log_aggr=1):
    model.train()

    sum_epoch_loss = 0

    start = time.time()
    for i, (uids, iids, labels, u_items, u_users, u_users_items, i_users,u_user_group, u_user_item_group) in tqdm(enumerate(train_loader), total=len(train_loader)):
        uids = uids.to(device)
        iids = iids.to(device)
        labels = labels.to(device)
        u_items = u_items.to(device)
        u_users = u_users.to(device)
        u_users_items = u_users_items.to(device)
        i_users = i_users.to(device)
        u_user_group =u_user_group.to(device)#### added 
        u_user_item_group = u_user_item_group.to(device)#### added 
        
        optimizer.zero_grad()
        outputs = model(uids, iids, u_items, u_users, u_users_items, i_users, u_user_group, u_user_item_group)#### changed

        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step() 

        loss_val = loss.item()
        sum_epoch_loss += loss_val

        iter_num = epoch * len(train_loader) + i + 1

        if i % log_aggr == 0:
            print('[TRAIN WWW] epoch %d/%d batch loss: %.4f (avg %.4f) (%.2f im/s)'
                % (epoch + 1, num_epochs, loss_val, sum_epoch_loss / (i + 1),
                  len(uids) / (time.time() - start)))

        start = time.time()



def validate(valid_loader, model):
    """
    Auxiliary function for epoch validate/test

    Running GraphRec model over given data loader (validation set loader / test set loader). 

    Returns mae and rmse score for given data.
    """
    model.eval()
    errors = []
    with torch.no_grad():
        for uids, iids, labels, u_items, u_users, u_users_items, i_users,u_user_group, u_user_item_group in tqdm(valid_loader):
            uids = uids.to(device)
            iids = iids.to(device)
            labels = labels.to(device)
            u_items = u_items.to(device)
            u_users = u_users.to(device)
            u_users_items = u_users_items.to(device)
            i_users = i_users.to(device)
            u_user_group =u_user_group.to(device)#### added 
            u_user_item_group = u_user_item_group.to(device)#### added 

            preds = model(uids, iids, u_items, u_users, u_users_items, i_users, u_user_group, u_user_item_group) #### changed
            error = torch.abs(preds.squeeze(1) - labels)
            errors.extend(error.data.cpu().numpy().tolist())
    
    mae = np.mean(errors)
    rmse = np.sqrt(np.mean(np.power(errors, 2)))
    return mae, rmse



# RUN training/testing
main(test = False)